In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import seaborn as sns
from statsmodels.graphics.regressionplots import *
import statsmodels.api as sm
from statsmodels.formula.api import ols

# special matplotlib argument for improved plots
from matplotlib import rcParams
sns.set_style("whitegrid")
sns.set_context("paper")
%matplotlib inline

In [2]:
atlas = pd.read_csv('/Users/rishigummakonda/Documents/Springboard/capstone/data/atlas_data/atlas.csv')
t_z= pd.read_csv('/Users/rishigummakonda/Documents/Springboard/capstone/data/TRACT_ZIP_032012.csv')

In [3]:
atlas.fillna(0,inplace=True)

In [4]:
nyc_atlas=pd.DataFrame(atlas[atlas['czname']=='New York'])
la_atlas=atlas[atlas['czname']=='Los Angeles']
denver_atlas= atlas[atlas['czname']=='Denver']
nashville_atlas= atlas[atlas['czname']=='Nashville']

In [5]:
nyc_atlas.reset_index(drop=True, inplace=True)
la_atlas.reset_index(drop=True,inplace=True)
denver_atlas.reset_index(drop=True,inplace=True)
nashville_atlas.reset_index(drop=True,inplace=True)

In [6]:
nyc_tracts=list(str(nyc_atlas['tract']))
la_tracts=list(str(la_atlas['tract']))
denver_tracts=list(str(denver_atlas['tract']))
nashville_tracts=list(str(nashville_atlas['tract']))

In [7]:
newyork_atlas = atlas[atlas['state']==36]
ca_atlas =atlas[atlas['state']==4]
co_atlas =atlas[atlas['state']==8]
tn_atlas =atlas[atlas['state']==47]

In [8]:
newyork_atlas.reset_index(drop=True, inplace=True)
ca_atlas.reset_index(drop=True, inplace=True)
co_atlas.reset_index(drop=True, inplace=True)
tn_atlas.reset_index(drop=True, inplace=True)

In [9]:
t_z.drop(['RES_RATIO','BUS_RATIO','OTH_RATIO','TOT_RATIO'], axis=1, inplace=True)

In [10]:
ny_zips = t_z[t_z['TRACT'].astype(str).str.startswith('36')]
ca_zips = t_z[t_z['TRACT'].astype(str).str.startswith('4')]
co_zips = t_z[t_z['TRACT'].astype(str).str.startswith('8')]
tn_zips = t_z[t_z['TRACT'].astype(str).str.startswith('47')]
print(ny_zips.shape)
print(ca_zips.shape)
print(co_zips.shape)
print(tn_zips.shape)

(10123, 2)
(33534, 2)
(2596, 2)
(3576, 2)


In [11]:
list_county_tract=[]
for tract in ny_zips['TRACT']:
    county_tract=str(tract)[2:]
    list_county_tract.append(county_tract)
ny_zips.insert(1, "county and tract", list_county_tract, True)

In [12]:
ny_zips

,TRACT,county and tract,ZIP
91527,36001000100,001000100,12207
91528,36001000100,001000100,12204
91529,36001000200,001000200,12204
91530,36001000200,001000200,12207
91531,36001000200,001000200,12210
...,...,...,...
101645,36123150400,123150400,14418
101646,36123150400,123150400,14478
101647,36123150500,123150500,14527
101648,36123150500,123150500,14837


In [13]:
list_of_countys=[]
for county_tract in ny_zips['county and tract']:
    county=str(county_tract)[:3]
    list_of_countys.append(int(county))
ny_zips.insert(2, 'county',list_of_countys,True)

In [14]:
ny_zips

,TRACT,county and tract,county,ZIP
91527,36001000100,001000100,1,12207
91528,36001000100,001000100,1,12204
91529,36001000200,001000200,1,12204
91530,36001000200,001000200,1,12207
91531,36001000200,001000200,1,12210
...,...,...,...,...
101645,36123150400,123150400,123,14418
101646,36123150400,123150400,123,14478
101647,36123150500,123150500,123,14527
101648,36123150500,123150500,123,14837


In [15]:
list_of_tracts=[]
for county_tract in ny_zips['county and tract']:
    tract=str(county_tract)[-6:]
    list_of_tracts.append(int(tract))
ny_zips.insert(3, 'tract',list_of_tracts,True)

In [16]:
ny_zips

,TRACT,county and tract,county,tract,ZIP
91527,36001000100,001000100,1,100,12207
91528,36001000100,001000100,1,100,12204
91529,36001000200,001000200,1,200,12204
91530,36001000200,001000200,1,200,12207
91531,36001000200,001000200,1,200,12210
...,...,...,...,...,...
101645,36123150400,123150400,123,150400,14418
101646,36123150400,123150400,123,150400,14478
101647,36123150500,123150500,123,150500,14527
101648,36123150500,123150500,123,150500,14837


In [17]:
list_of_state=[]
for county_tract in ny_zips['TRACT']:
    state=str(county_tract)[0:2]
    list_of_state.append(int(state))
ny_zips.insert(2, 'state',list_of_state,True)

In [18]:
ny_zips

,TRACT,county and tract,state,county,tract,ZIP
91527,36001000100,001000100,36,1,100,12207
91528,36001000100,001000100,36,1,100,12204
91529,36001000200,001000200,36,1,200,12204
91530,36001000200,001000200,36,1,200,12207
91531,36001000200,001000200,36,1,200,12210
...,...,...,...,...,...,...
101645,36123150400,123150400,36,123,150400,14418
101646,36123150400,123150400,36,123,150400,14478
101647,36123150500,123150500,36,123,150500,14527
101648,36123150500,123150500,36,123,150500,14837


In [19]:
def split_

SyntaxError: invalid syntax (<ipython-input-19-2237be42dab8>, line 1)

you have to write the above function to split it up

then u have to write a function which outputs a dict of zips when the tracts match

then u have to write the code to find the distance between one of the zip codes within the most populus tract from one of the zips in each of the tracts.

then once you have the distance you have to go back and match it up with income data